# Pruebas

In [ ]:
pip install google-api-python-client

In [ ]:
from googleapiclient.discovery import build

## Comments

In [29]:
API_KEY = 'API KEY'

Enter a keyword: Messi
Enter start year: 2016
Enter end year: 2017


HttpError: ignored

In [ ]:
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

def get_video_comments(video_id):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

    comments = []

    next_page_token = None
    while True:
        comments_response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            textFormat='plainText',
            maxResults=100,
            pageToken=next_page_token
        ).execute()

        for comment_item in comments_response.get('items', []):
            comment = comment_item['snippet']['topLevelComment']['snippet']
            comment_text = comment['textDisplay']
            comment_publish_time = comment['publishedAt']
            comments.append((comment_publish_time, comment_text))

        next_page_token = comments_response.get('nextPageToken')
        if not next_page_token:
            break

    return comments

def get_video_info(video_id):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

    video_response = youtube.videos().list(
        part='statistics',
        id=video_id
    ).execute()

    video_info = video_response.get('items', [])[0]['statistics']
    views_count = video_info['viewCount']
    comments_count = video_info['commentCount']

    return views_count, comments_count

def get_videos_in_time_range(keyword, start_year, end_year, max_results=10):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

    search_response = youtube.search().list(
        q=keyword,
        type='video',
        part='id,snippet',
        maxResults=max_results,
        publishedAfter=f'{start_year}-01-01T00:00:00Z',
        publishedBefore=f'{end_year+1}-01-01T00:00:00Z'
    ).execute()

    video_data = []
    for search_result in search_response.get('items', []):
        video_id = search_result['id']['videoId']
        video_title = search_result['snippet']['title']
        views_count, comments_count = get_video_info(video_id)
        video_data.append({'video_id': video_id, 'title': video_title, 'views': views_count, 'comments': comments_count})

    return video_data

if __name__ == '__main__':
    keyword = input("Enter a keyword: ")
    start_year = int(input("Enter start year: "))
    end_year = int(input("Enter end year: "))
    video_data = get_videos_in_time_range(keyword, start_year, end_year)

    if video_data:
        print("Video Data:")
        for data in video_data:
            print(f"Video ID: {data['video_id']}")
            print(f"Title: {data['title']}")
            print(f"Views: {data['views']}")
            print(f"Comments: {data['comments']}")
            print("=" * 30)

            video_comments = get_video_comments(data['video_id'])
            if video_comments:
                with open(f"{data['video_id']}_comments.txt", "w", encoding="utf-8") as file:
                    for publish_time, comment in video_comments:
                        file.write(f"Publish Time: {publish_time}\n")
                        file.write(f"Comment: {comment}\n\n")

                print(f"Comments exported to {data['video_id']}_comments.txt")

            print("=" * 30)
    else:
        print("No videos found.")

### Total Video Count by Year

In [7]:
from googleapiclient.discovery import build
from collections import defaultdict

YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

def get_total_video_count_by_year(keyword, start_year, end_year):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

    video_count_by_year = defaultdict(int)

    for year in range(start_year, end_year + 1):
        search_response = youtube.search().list(
            q=keyword,
            type='video',
            part='id',
            maxResults=1,
            publishedAfter=f'{year}-01-01T00:00:00Z',
            publishedBefore=f'{year+1}-01-01T00:00:00Z'
        ).execute()

        total_results = search_response.get('pageInfo', {}).get('totalResults', 0)
        video_count_by_year[year] = total_results

    return video_count_by_year

if __name__ == '__main__':
    keyword = input("Enter a keyword: ")
    start_year = int(input("Enter start year: "))
    end_year = int(input("Enter end year: "))

    video_count_by_year = get_total_video_count_by_year(keyword, start_year, end_year)

    print("Total Video Count by Year:")
    for year, count in video_count_by_year.items():
        print(f"{year}: {count} videos")


Enter a keyword: Ruben Dias
Enter start year: 2010
Enter end year: 2022
Total Video Count by Year:
2010: 9141 videos
2011: 11291 videos
2012: 15995 videos
2013: 21763 videos
2014: 77407 videos
2015: 68197 videos
2016: 97334 videos
2017: 150404 videos
2018: 263526 videos
2019: 430274 videos
2020: 742745 videos
2021: 1000000 videos
2022: 1000000 videos
